### Coursework 4

In [ ]:
# Each block was called from a run.sh script individually after adjusting its parameters
# The results were saved in txt files

In [ ]:
# Baseline training

In [ ]:
import os
import tensorflow as tf
import datetime
from collections import OrderedDict
import numpy as np
import pickle as pcl
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
from collections import OrderedDict
#matplotlib inline
#from IPython.display import Image


assert 'MLP_DATA_DIR' in os.environ, (
    'An environment variable MLP_DATA_DIR must be set to the path containing'
    ' MLP data before running script.')
assert 'OUTPUT_DIR' in os.environ, (
    'An environment variable OUTPUT_DIR must be set to the path to write'
    ' output to before running script.')

# Run a standard CNN to get a model that overfits

models = {} # store everything

# Helping functions

def set_wb(wb,mp):
    p = 32/(mp*2)
    weights = {
        'wc1': weight_variable([5, 5, 3, wb]),
        'wc2': weight_variable([5, 5, wb, wb*2]),
        'wc3': weight_variable([5, 5, wb*2, wb*2]),
        'wd1': weight_variable([p*p*(wb*2), wb**3]),
        'out': weight_variable([wb**3, 10])
    }

    biases = {
        'bc1': bias_variable([wb]),
        'bc2': bias_variable([wb*2]),
        'bc3': bias_variable([wb*2]),
        'bd1': bias_variable([wb**3]),
        'out': bias_variable([10])
    }
    return weights, biases

def write_to_file(newfile, run_info):
    with open(newfile,'w') as outfile:
        pcl.dump(run_info,outfile)

def read_from_file(newfile):
    run_info = pcl.load(open(newfile,'r'))
    return run_info

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# model: CNN/RELU>CNN/RELU>MP>CNN/RELU>MP>FC

def conv_net(x, weights, biases):
    x = tf.reshape(x, shape=[-1, 32, 32, 3])
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    #conv12 = conv2d(conv1, weights['wc2'], biases['bc2'])
    #conv13 = conv2d(conv12, weights['wc3'], biases['bc3'])
    #conv14 = conv2d(conv13, weights['wc3'], biases['bc3'])

    # Max Pooling
    #conv1 = maxpool2d(conv1, k=2)
    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling
    conv2 = maxpool2d(conv2, k=2)
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling
    conv3 = maxpool2d(conv3, k=2)
    # Fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


## session

num_epoch = 30 
fname = "baseK5-"

train_data = CIFAR10DataProvider('train', batch_size=50)
valid_data = CIFAR10DataProvider('valid', batch_size=50)
valid_inputs = valid_data.inputs
valid_targets = valid_data.to_one_of_k(valid_data.targets)

ks = [32]
for k in ks:
    tf.reset_default_graph()
    weightsk, biasesk = set_wb(k,2)
    
    # define model graph
    with tf.name_scope('data'):
        inputs = tf.placeholder(tf.float32, [None, 3072], name='inputs')
        targets = tf.placeholder(tf.float32, [None, 10], name='targets')
    #with tf.name_scope('parameters'):
    #    weights = tf.Variable(tf.zeros([3072, 10]), name='weights')
    #    biases = tf.Variable(tf.zeros([10]), name='biases')
    with tf.name_scope('model'):
        outputs = conv_net(inputs, weightsk, biasesk)
    with tf.name_scope('error'):
        error = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(outputs,targets))
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer().minimize(error)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(tf.equal(
            tf.argmax(outputs, 1), tf.argmax(targets, 1)), tf.float32))

    train_accuracy = np.zeros(num_epoch)
    train_error = np.zeros(num_epoch)
    valid_accuracy = np.zeros(num_epoch)
    valid_error = np.zeros(num_epoch)

    name = str(k)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    step = 0
    for e in range(num_epoch):
        for b, (input_batch, target_batch) in enumerate(train_data):

            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy],
                feed_dict={inputs: input_batch, targets: target_batch})

            train_error[e] += batch_error
            train_accuracy[e] += batch_acc
            step += 1
        
        train_error[e] /= train_data.num_batches
        train_accuracy[e] /= train_data.num_batches

        valid_error[e], valid_accuracy[e] = sess.run(
            [error, accuracy],
            feed_dict={inputs: valid_inputs, targets: valid_targets})

    stats = (train_error, train_accuracy, valid_error, valid_accuracy)
    
    ffname = fname+name+"_10.txt"
    write_to_file(ffname,stats)
sess.close()

In [ ]:
# Dropout on the fully connected

In [ ]:
import os
import tensorflow as tf
import datetime
from collections import OrderedDict
import numpy as np
import pickle as pcl
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
from collections import OrderedDict
#matplotlib inline
#from IPython.display import Image


assert 'MLP_DATA_DIR' in os.environ, (
    'An environment variable MLP_DATA_DIR must be set to the path containing'
    ' MLP data before running script.')

assert 'OUTPUT_DIR' in os.environ, (
    'An environment variable OUTPUT_DIR must be set to the path to write'
    ' output to before running script.')

# Run a standard CNN to get a model that overfits

# Helping functions

def set_wb(wb,mp):
    weights = {
        'wc1': weight_variable([2, 2, 3, 8]),
        'wc2': weight_variable([2, 2, 8, 16]),
        'wc3': weight_variable([2, 2, 16, 16]),
        'wd1': weight_variable([8*8*16, 8**3]),
        'out': weight_variable([8**3, 10])
    }

    biases = {
        'bc1': bias_variable([8]),
        'bc2': bias_variable([16]),
        'bc3': bias_variable([16]),
        'bd1': bias_variable([8**3]),
        'out': bias_variable([10])
    }
    return weights, biases

def write_to_file(newfile, run_info):
    with open(newfile,'w') as outfile:
        pcl.dump(run_info,outfile)

def read_from_file(newfile):
    run_info = pcl.load(open(newfile,'r'))
    return run_info

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# model: CNN/RELU>CNN/RELU>MP>CNN/RELU>MP>FC
# DROP ONCE
def conv_net(x, weights, biases, drop):
    x = tf.reshape(x, shape=[-1, 32, 32, 3])
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv12 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv13 = conv2d(conv12, weights['wc3'], biases['bc3'])
    conv14 = conv2d(conv13, weights['wc3'], biases['bc3'])
    # Max Pooling
    #conv1 = maxpool2d(conv1, k=2)
    # Convolution Layer
    conv2 = conv2d(conv14, weights['wc3'], biases['bc3'])
    # Max Pooling
    conv2 = maxpool2d(conv2, k=2)
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling
    conv3 = maxpool2d(conv3, k=2)
    # Fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, drop)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


## session

num_epoch = 30 
fname = "base6CNN-drop-fc"

train_data = CIFAR10DataProvider('train', batch_size=50)
valid_data = CIFAR10DataProvider('valid', batch_size=50)
valid_inputs = valid_data.inputs
valid_targets = valid_data.to_one_of_k(valid_data.targets)
keep_prob = tf.placeholder(tf.float32)

pb = [0.1,0.25,0.5,0.75,0.9]
for p in pb:
    tf.reset_default_graph()
    weightsk, biasesk = set_wb(8,2)
    
    # define model graph
    with tf.name_scope('data'):
        inputs = tf.placeholder(tf.float32, [None, 3072], name='inputs')
        targets = tf.placeholder(tf.float32, [None, 10], name='targets')
    #with tf.name_scope('parameters'):
    #    weights = tf.Variable(tf.zeros([3072, 10]), name='weights')
    #    biases = tf.Variable(tf.zeros([10]), name='biases')
    with tf.name_scope('model'):
        outputs = conv_net(inputs, weightsk, biasesk,p)
    with tf.name_scope('error'):
        error = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(outputs,targets))
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer().minimize(error)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(tf.equal(
            tf.argmax(outputs, 1), tf.argmax(targets, 1)), tf.float32))

    train_accuracy = np.zeros(num_epoch)
    train_error = np.zeros(num_epoch)
    valid_accuracy = np.zeros(num_epoch)
    valid_error = np.zeros(num_epoch)

    name = str(p)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    step = 0
    for e in range(num_epoch):
        for b, (input_batch, target_batch) in enumerate(train_data):

            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy],
                feed_dict={inputs: input_batch, targets: target_batch})

            train_error[e] += batch_error
            train_accuracy[e] += batch_acc
            step += 1
        
        train_error[e] /= train_data.num_batches
        train_accuracy[e] /= train_data.num_batches

        valid_error[e], valid_accuracy[e] = sess.run(
            [error, accuracy],
            feed_dict={inputs: valid_inputs, targets: valid_targets})

    stats = (train_error, train_accuracy, valid_error, valid_accuracy)
    ffname = fname+name+"_10.txt"
    write_to_file(ffname,stats)
sess.close()

In [ ]:
# Dropout on all

In [ ]:
import os
import tensorflow as tf
import datetime
from collections import OrderedDict
import numpy as np
import pickle as pcl
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
from collections import OrderedDict
#matplotlib inline
#from IPython.display import Image


assert 'MLP_DATA_DIR' in os.environ, (
    'An environment variable MLP_DATA_DIR must be set to the path containing'
    ' MLP data before running script.')

assert 'OUTPUT_DIR' in os.environ, (
    'An environment variable OUTPUT_DIR must be set to the path to write'
    ' output to before running script.')

# Run a standard CNN to get a model that overfits

# Helping functions

def set_wb(wb,mp):
    weights = {
        'wc1': weight_variable([5, 5, 3, 8]),
        'wc2': weight_variable([5, 5, 8, 16]),
        'wc3': weight_variable([5, 5, 16, 16]),
        'wd1': weight_variable([8*8*16, 8**3]),
        'out': weight_variable([8**3, 10])
    }

    biases = {
        'bc1': bias_variable([8]),
        'bc2': bias_variable([16]),
        'bc3': bias_variable([16]),
        'bd1': bias_variable([8**3]),
        'out': bias_variable([10])
    }
    return weights, biases

def write_to_file(newfile, run_info):
    with open(newfile,'w') as outfile:
        pcl.dump(run_info,outfile)

def read_from_file(newfile):
    run_info = pcl.load(open(newfile,'r'))
    return run_info

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# model: CNN/RELU>CNN/RELU>MP>CNN/RELU>MP>FC
# DROP ONCE
def conv_net(x, weights, biases, drop):
    x = tf.reshape(x, shape=[-1, 32, 32, 3])
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = tf.nn.dropout(conv1,drop)
    #conv12 = conv2d(conv1, weights['wc2'], biases['bc2'])
    #conv13 = conv2d(conv12, weights['wc3'], biases['bc3'])
    #conv14 = conv2d(conv13, weights['wc3'], biases['bc3'])
    # Max Pooling
    #conv1 = maxpool2d(conv1, k=2)
    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling
    conv2 = tf.nn.dropout(conv2,drop)
    conv2 = maxpool2d(conv2, k=2)
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling
    conv3 = tf.nn.dropout(conv3,drop)
    conv3 = maxpool2d(conv3, k=2)
    # Fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, drop)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


## session

num_epoch = 30 
fname = "baseK5-drop"

train_data = CIFAR10DataProvider('train', batch_size=50)
valid_data = CIFAR10DataProvider('valid', batch_size=50)
valid_inputs = valid_data.inputs
valid_targets = valid_data.to_one_of_k(valid_data.targets)
keep_prob = tf.placeholder(tf.float32)

pb = [0.1,0.25,0.5,0.75,0.9]
for p in pb:
    tf.reset_default_graph()
    weightsk, biasesk = set_wb(8,2)
    
    # define model graph
    with tf.name_scope('data'):
        inputs = tf.placeholder(tf.float32, [None, 3072], name='inputs')
        targets = tf.placeholder(tf.float32, [None, 10], name='targets')
    #with tf.name_scope('parameters'):
    #    weights = tf.Variable(tf.zeros([3072, 10]), name='weights')
    #    biases = tf.Variable(tf.zeros([10]), name='biases')
    with tf.name_scope('model'):
        outputs = conv_net(inputs, weightsk, biasesk,p)
    with tf.name_scope('error'):
        error = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(outputs,targets))
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer().minimize(error)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(tf.equal(
            tf.argmax(outputs, 1), tf.argmax(targets, 1)), tf.float32))

    train_accuracy = np.zeros(num_epoch)
    train_error = np.zeros(num_epoch)
    valid_accuracy = np.zeros(num_epoch)
    valid_error = np.zeros(num_epoch)

    name = str(p)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    step = 0
    for e in range(num_epoch):
        for b, (input_batch, target_batch) in enumerate(train_data):

            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy],
                feed_dict={inputs: input_batch, targets: target_batch})

            train_error[e] += batch_error
            train_accuracy[e] += batch_acc
            step += 1
        
        train_error[e] /= train_data.num_batches
        train_accuracy[e] /= train_data.num_batches

        valid_error[e], valid_accuracy[e] = sess.run(
            [error, accuracy],
            feed_dict={inputs: valid_inputs, targets: valid_targets})

    stats = (train_error, train_accuracy, valid_error, valid_accuracy)
    ffname = fname+name+"_10.txt"
    write_to_file(ffname,stats)
sess.close()

In [ ]:
# L2 regularization

In [ ]:
import os
import tensorflow as tf
import datetime
from collections import OrderedDict
import numpy as np
import pickle as pcl
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
from collections import OrderedDict
#matplotlib inline
#from IPython.display import Image


assert 'MLP_DATA_DIR' in os.environ, (
    'An environment variable MLP_DATA_DIR must be set to the path containing'
    ' MLP data before running script.')

assert 'OUTPUT_DIR' in os.environ, (
    'An environment variable OUTPUT_DIR must be set to the path to write'
    ' output to before running script.')

# Run a standard CNN to get a model that overfits

# Helping functions

def set_wb(wb,mp):
    weights = {
        'wc1': weight_variable([2, 2, 3, 8]),
        'wc2': weight_variable([2, 2, 8, 16]),
        'wc3': weight_variable([2, 2, 16, 16]),
        'wd1': weight_variable([8*8*16, 8**3]),
        'out': weight_variable([8**3, 10])
    }

    biases = {
        'bc1': bias_variable([8]),
        'bc2': bias_variable([16]),
        'bc3': bias_variable([16]),
        'bd1': bias_variable([8**3]),
        'out': bias_variable([10])
    }
    return weights, biases

def write_to_file(newfile, run_info):
    with open(newfile,'w') as outfile:
        pcl.dump(run_info,outfile)

def read_from_file(newfile):
    run_info = pcl.load(open(newfile,'r'))
    return run_info

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# model: CNN/RELU>CNN/RELU>MP>CNN/RELU>MP>FC
# DROP ONCE
def conv_net(x, weights, biases, penalty):
    x = tf.reshape(x, shape=[-1, 32, 32, 3])
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    l1 = penalty*tf.nn.l2_loss(weights['wc1'])
    conv12 = conv2d(conv1, weights['wc2'], biases['bc2'])
    l12 = penalty*tf.nn.l2_loss(weights['wc2'])
    conv13 = conv2d(conv12, weights['wc3'], biases['bc3'])
    l13 = penalty*tf.nn.l2_loss(weights['wc3'])
    conv14 = conv2d(conv13, weights['wc3'], biases['bc3'])
    l14 = penalty*tf.nn.l2_loss(weights['wc3'])
    # Max Pooling
    #conv1 = maxpool2d(conv1, k=2)
    # Convolution Layer
    conv2 = conv2d(conv14, weights['wc3'], biases['bc3'])
	#conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    l2 = penalty*tf.nn.l2_loss(weights['wc3'])
    # Max Pooling
    conv2 = maxpool2d(conv2, k=2)
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    l3 = penalty*tf.nn.l2_loss(weights['wc3'])
    # Max Pooling
    conv3 = maxpool2d(conv3, k=2)
    # Fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    l4 = penalty*tf.nn.l2_loss(weights['wd1'])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    #fc1 = tf.nn.dropout(fc1, drop)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    loss = l1 + l12 + l13 + l14 + l2 + l3 + l4
    return out,loss


## session

num_epoch = 30 
fname = "base6CNN-L2"

train_data = CIFAR10DataProvider('train', batch_size=50)
valid_data = CIFAR10DataProvider('valid', batch_size=50)
valid_inputs = valid_data.inputs
valid_targets = valid_data.to_one_of_k(valid_data.targets)
keep_prob = tf.placeholder(tf.float32)

penalty = [0.1,0.01,0.001,0.0001]
for p in penalty:
    tf.reset_default_graph()
    weightsk, biasesk = set_wb(8,2)
    
    # define model graph
    with tf.name_scope('data'):
        inputs = tf.placeholder(tf.float32, [None, 3072], name='inputs')
        targets = tf.placeholder(tf.float32, [None, 10], name='targets')
    #with tf.name_scope('parameters'):
    #    weights = tf.Variable(tf.zeros([3072, 10]), name='weights')
    #    biases = tf.Variable(tf.zeros([10]), name='biases')
    with tf.name_scope('model'):
        outputs,loss = conv_net(inputs, weightsk, biasesk,p)
    with tf.name_scope('error'):
        error = tf.reduce_mean(tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(outputs, targets)) + loss)
    with tf.name_scope('error_val'):
        error_val = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(outputs, targets))
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer().minimize(error)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(tf.equal(
            tf.argmax(outputs, 1), tf.argmax(targets, 1)), tf.float32))

    train_accuracy = np.zeros(num_epoch)
    train_error = np.zeros(num_epoch)
    valid_accuracy = np.zeros(num_epoch)
    valid_error = np.zeros(num_epoch)

    name = str(p)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    step = 0
    for e in range(num_epoch):
        for b, (input_batch, target_batch) in enumerate(train_data):

            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy],
                feed_dict={inputs: input_batch, targets: target_batch})

            train_error[e] += batch_error
            train_accuracy[e] += batch_acc
            step += 1
        
        train_error[e] /= train_data.num_batches
        train_accuracy[e] /= train_data.num_batches

        valid_error[e], valid_accuracy[e] = sess.run(
            [error_val, accuracy],
            feed_dict={inputs: valid_inputs, targets: valid_targets})

    stats = (train_error, train_accuracy, valid_error, valid_accuracy)
    ffname = fname+name+"_10.txt"
    write_to_file(ffname,stats)
sess.close()

In [ ]:
# Data augmentation

In [ ]:
import os
import tensorflow as tf
import datetime
from collections import OrderedDict
import numpy as np
import pickle as pcl
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
import random
from collections import OrderedDict
#matplotlib inline
#from IPython.display import Image


assert 'MLP_DATA_DIR' in os.environ, (
    'An environment variable MLP_DATA_DIR must be set to the path containing'
    ' MLP data before running script.')
assert 'OUTPUT_DIR' in os.environ, (
    'An environment variable OUTPUT_DIR must be set to the path to write'
    ' output to before running script.')

def set_wb(wb,mp):
    p = 32/(mp*2)
    weights = {
        'wc1': weight_variable([5, 5, 3, wb]),
        'wc2': weight_variable([5, 5, wb, wb*2]),
        'wc3': weight_variable([5, 5, wb*2, wb*2]),
        'wd1': weight_variable([p*p*(wb*2), wb**3]),
        'out': weight_variable([wb**3, 10])
    }

    biases = {
        'bc1': bias_variable([wb]),
        'bc2': bias_variable([wb*2]),
        'bc3': bias_variable([wb*2]),
        'bd1': bias_variable([wb**3]),
        'out': bias_variable([10])
    }
    return weights, biases

def write_to_file(newfile, run_info):
    with open(newfile,'w') as outfile:
        pcl.dump(run_info,outfile)

def read_from_file(newfile):
    run_info = pcl.load(open(newfile,'r'))
    return run_info

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# model: CNN/RELU>CNN/RELU>MP>CNN/RELU>MP>FC

def conv_net(x, weights, biases):
    x = tf.reshape(x, shape=[-1, 32, 32, 3])
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling
    #conv1 = maxpool2d(conv1, k=2)
    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling
    conv2 = maxpool2d(conv2, k=2)
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling
    conv3 = maxpool2d(conv3, k=2)
    # Fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


## TRANFORMATIONS

def pre_process_image(image):
    image = tf.reshape(image,[32,32,3])

    choice = random.randint(0, 3)
    if (choice == 0):
        image = tf.image.random_flip_left_right(image)
        #image = tf.image.random_flip_up_down(image)
        # Randomly adjust hue, contrast and saturation.
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)

        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)

    return tf.reshape(image,[-1])

def pre_process_image_flip_ud(image):
    image = tf.reshape(image,[32,32,3])
    choice = random.randint(0, 3)
    if (choice == 0):
        image = tf.image.random_flip_up_down(image)
        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)

    return tf.reshape(image,[-1])

def pre_process_image_flip_lr(image):
    image = tf.reshape(image,[32,32,3])
    choice = random.randint(0, 3)
    if (choice == 0):
        image = tf.image.random_flip_left_right(image)
        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)

    return tf.reshape(image,[-1])

def pre_process_image_hue(image):
    image = tf.reshape(image,[32,32,3])
    # Randomly adjust hue, contrast and saturation.
    choice = random.randint(0, 3)
    if (choice == 0):
        image = tf.image.random_hue(image, max_delta=0.05)
        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)
    return tf.reshape(image,[-1])

def pre_process_image_contrast(image):
    image = tf.reshape(image,[32,32,3])

    choice = random.randint(0, 3)
    if (choice == 0):
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)

    return tf.reshape(image,[-1])

def pre_process_image_brightness(image):
    image = tf.reshape(image,[32,32,3])

    choice = random.randint(0, 3)
    if (choice == 0):
        image = tf.image.random_brightness(image, max_delta=0.2)
        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)

    return tf.reshape(image,[-1])

def pre_process_image_saturation(image):
    image = tf.reshape(image,[32,32,3])
    choice = random.randint(0, 3)
    if (choice == 0):
        # Randomly adjust hue, contrast and saturation.
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)
        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)

    return tf.reshape(image,[-1])

def pre_process(x, trans):
    if (trans == 'flip_ud'):
        x = tf.map_fn(lambda image: pre_process_image_flip_ud(image), x)
    elif (trans == 'flip_lr'):
        x = tf.map_fn(lambda image: pre_process_image_flip_lr(image), x)
    elif (trans == 'hue'):
        x = tf.map_fn(lambda image: pre_process_image_hue(image), x)
    elif (trans == 'contrast'):
        x = tf.map_fn(lambda image: pre_process_image_contrast(image), x)
    elif (trans == 'brightness'):
        x = tf.map_fn(lambda image: pre_process_image_brightness(image), x)
    elif (trans == 'saturation'):
        x = tf.map_fn(lambda image: pre_process_image_saturation(image), x)
    else:
        print('No transformation used')

    return x



## session

num_epoch = 30
fname = "baseDAK5-"

train_data = CIFAR10DataProvider('train', batch_size=50)
valid_data = CIFAR10DataProvider('valid', batch_size=50)
valid_inputs = valid_data.inputs
valid_targets = valid_data.to_one_of_k(valid_data.targets)

weightsk, biasesk = set_wb(8,2)

transf = ['flip_ud', 'flip_lr', 'hue','contrast','brightness','saturation']
for trans in transf:
    tf.reset_default_graph()
    weightsk, biasesk = set_wb(8,2)

    with tf.name_scope('data'):
        inputs = tf.placeholder(tf.float32, [None, 3072], name='inputs')
        targets = tf.placeholder(tf.float32, [None, 10], name='targets')
        preprocessed_inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1]], name='dinputs')
        distorted_inputs = pre_process(preprocessed_inputs, trans)
    with tf.name_scope('model'):
        outputs = conv_net(inputs, weightsk, biasesk)
    with tf.name_scope('error'):
        error = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(outputs,targets))
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer().minimize(error)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(tf.equal(
            tf.argmax(outputs, 1), tf.argmax(targets, 1)), tf.float32))

    train_accuracy = np.zeros(num_epoch)
    train_error = np.zeros(num_epoch)
    valid_accuracy = np.zeros(num_epoch)
    valid_error = np.zeros(num_epoch)

    name = str(trans)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    step = 0
    for e in range(num_epoch):
        for b, (input_batch, target_batch) in enumerate(train_data):
            # do train step with current batch
            input_batch_dist = sess.run(distorted_inputs, feed_dict={preprocessed_inputs: input_batch})
            _ = sess.run(
                [train_step],
                feed_dict={inputs: input_batch_dist, targets: target_batch})
            batch_error, batch_acc = sess.run(
                [error, accuracy],
                feed_dict={inputs: input_batch, targets: target_batch})
            train_error[e] += batch_error
            train_accuracy[e] += batch_acc
            step += 1
        train_error[e] /= train_data.num_batches
        train_accuracy[e] /= train_data.num_batches
        valid_error[e], valid_accuracy[e] = sess.run(
            [error, accuracy],
            feed_dict={inputs: valid_inputs, targets: valid_targets})

    stats = (train_error, train_accuracy, valid_error, valid_accuracy)
    ffname = fname+name+"_10.txt"
    write_to_file(ffname,stats)
sess.close()


In [ ]:
# The results were plotted with input the txt files